In [34]:
import sys; sys.path.insert(0,'..')
from contxt import *

In [35]:
path_epistolary='/Users/ryan/workspace/epistolary/'
path_epistolary_chars = os.path.join(path_epistolary,'chars')
path_epistolary_txt = os.path.join(path_epistolary,'txt')
path_epistolary_letters = os.path.join(path_epistolary,'letters')

In [36]:
# !ls {path_epistolary_txt}/chadwyck/Eighteenth-Century_Fiction/richards.01

In [37]:
chadwyck_metadata=pd.read_csv('/Users/ryan/lltk_data/corpora/chadwyck/metadata.csv').set_index('id')
# chadwyck_metadata

In [88]:
class EpistolaryText:
    def __init__(self, letter_fn, char_fn, text_dir, meta={}):
        self.letter_fn=letter_fn
        self.char_fn=char_fn
        self.text_dir=text_dir
        self.meta=meta
        
    @cached_property
    def _letter_df(self):
        return pd.read_excel(self.letter_fn) if '.xls' in self.letter_fn else pd.read_csv(self.letter_fn)

    @cached_property
    def _char_df(self):
        return pd.read_excel(self.char_fn) if '.xls' in self.char_fn else pd.read_csv(self.char_fn)
    
    @cached_property
    def _char_d(self): return dict(zip(self._char_df.char_tok, self._char_df.char_id))

    def get_char_id(self, char_tok):
        return self._char_d.get(char_tok,char_tok)
    
    def _letter_data(self, letter_df_row, score=True):
        outd={**self.meta, **letter_df_row}
        if 'sender_tok' in outd:
            outd['sender_id']=self.get_char_id(outd['sender_tok'])
        if 'recip_tok' in outd:
            outd['recip_id']=self.get_char_id(outd['recip_tok'])
        
        # read
        lid = outd['letter_id'] = outd['id']
        path1 = os.path.join(self.text_dir, lid, 'text.txt')
        path2 = os.path.join(self.text_dir, letter_df_row.id_orig+'.txt')
        for path in [path1,path2]:
            if os.path.exists(path):
                with open(path) as f: txt=f.read()
                outd['psg_txt'] = txt
                break
        
        return outd
    
    
    @cached_property
    def scanned_letters(self):
        return pd.DataFrame([self._letter_data(row) for i,row in self._letter_df.iterrows()]).fillna('')
    
    @cached_property
    def scored_letters(self):
        return score_absconc_psg(self.scanned_letters)



In [117]:
clarissa = EpistolaryText(
    letter_fn=f'/Users/ryan/workspace/epistolary/letters/chadwyck/Eighteenth-Century_Fiction/metadata-clarissa.xls',
    # char_fn=f'/Users/ryan/workspace/epistolary/chars/chadwyck/Eighteenth-Century_Fiction/richards.01/tok2id.anno.xls',
    char_fn=f'/Users/ryan/workspace/epistolary/chars/chadwyck/Eighteenth-Century_Fiction/richards.01/names3.xlsx',
    text_dir='/Users/ryan/github/abslithist2023/texts/richards.01/letters/texts',
    meta=chadwyck_metadata.loc['Eighteenth-Century_Fiction/richards.01']
)
clarissa.scanned_letters

,author,title,year,idref,pub,type,name,nation,medium,subcorpus,...,date,place,notes,date_ner,date_time,place_ner,sender_id,recip_id,letter_id,psg_txt
0,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Jan. 10,,,Jan. 10,2022-01-10 00:00:00,,AnnaHowe,ClarissaHarlowe,L001a,"Miss Anna Howe, To Miss Clarissa Harlowe.\n\n\..."
1,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Jan. 13,Harlowe-Place,,Jan. 13,2022-01-13 00:00:00,Harlowe-Place,ClarissaHarlowe,AnnaHowe,L002a,"Miss Clarissa Harlowe, To Miss Howe.\n\n\nHarl..."
2,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Jan. 13-14,Harlowe-Place,,Jan. 13 | 14,2014-01-13 00:00:00,,ClarissaHarlowe,AnnaHowe,L003a,"Miss Clarissa Harlowe, To Miss Howe.\n\n\nJan...."
3,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,,Harlowe-Place,,,,Scotland,ClarissaHarlowe,AnnaHowe,L004a,"Miss Clarissa Harlowe, To Miss Howe.\n\n\nThis..."
4,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Jan. 20,Harlowe-Place,,Jan. 20,2022-01-20 00:00:00,,ClarissaHarlowe,AnnaHowe,L005a,"Miss Clarissa Harlowe, To Miss Howe.\n\n\nJan...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Nov. 28 | Dec. 9,Trent,,Nov. 28 | Dec. 9,None,Trent,RobertLovelace,JohnBelford,L525b,"Mr. Lovelace, To John Belford, Esq; | * * | * ..."
621,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Nov. 21 | Dec. 2 | four days,Munich | Florence,,Nov. 21 | Dec. 2 | four days,None,Munich | Florence,WilliamMorden,RobertLovelace,L525c,"Munich, Nov. 21. ⪆⪆Dec. 2.\n\nSir, Your letter..."
622,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Nov. 25 | Dec. 6,Vienna,,Nov. 25 | Dec. 6,None,Vienna,RobertLovelace,WilliamMorden,L525d,"Vienna, Nov. 25. ⪆⪆Dec. 6.\n\nSir, I have this..."
623,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Dec. 3 - 14 | the Day,,,Dec. 3 - 14 | the Day,None,,RobertLovelace,JohnBelford,L526a,"Mr. Lovelace, To John Belford, Esq;\n\n\nTrent..."


In [118]:
pamela = EpistolaryText(
    letter_fn=f'/Users/ryan/workspace/epistolary/letters/chadwyck/Eighteenth-Century_Fiction/metadata-pamela.xls',
    char_fn=f'/Users/ryan/workspace/epistolary/chars/chadwyck/Eighteenth-Century_Fiction/richards.04/tok2id.anno.csv',
    text_dir=f'/Users/ryan/workspace/epistolary/txt/chadwyck/Eighteenth-Century_Fiction/richards.04',
    meta=chadwyck_metadata.loc['Eighteenth-Century_Fiction/richards.04']
)
pamela.scanned_letters.drop_duplicates('id_orig')

,author,title,year,idref,pub,type,name,nation,medium,subcorpus,...,sender_tok,recip_tok,date_ner,date_time,num_words,notes,sender_id,recip_id,letter_id,psg_txt
0,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,x,x,,,382,,x,x,L001a,
1,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,x,x,,,585,,x,x,L002a,
2,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,x,x,,,1139,,x,x,L003a,
3,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Pamela,John Andrews; Elizabeth Andrews,,,907,,Pamela,John Andrews; Elizabeth Andrews,L004a,"Dear Father and Mother, | I have great Trouble..."
4,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,John Andrews; Elizabeth Andrews,Pamela,,,715,,John Andrews; Elizabeth Andrews,Pamela,L005a,"Dear Pamela, | Your Letter was indeed a great ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Pamela,Miss Darnford,,,1686,,Pamela,MissDarnford,L207a,"My dear Lady G. | The Remarks which, your Cous..."
245,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Pamela,Miss Darnford,,,9336,,Pamela,MissDarnford,L208b,My dear Lady G. | I will chearfully cause to b...
247,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,Pamela,Miss Darnford,,,5133,,Pamela,MissDarnford,L209a,My dear Lady G. | You desired me to send you a...
248,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,x,x,,,756,,x,x,L210a,


In [119]:
richardson = pd.concat([pamela.scanned_letters.drop_duplicates('id_orig'), clarissa.scanned_letters])
richardson_scored = score_absconc_psg(richardson)
richardson_scored

,author,title,year,idref,pub,type,name,nation,medium,subcorpus,...,date,place,place_ner,psg_num_tokens,psg_num_words,absconc_words_median,absconc_words_mean,absconc_words_stdev,absconc_words_num,absconc_words_str
psg_i,,,,,,,,,,,,,,,,,,,,,
0,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,NaN,NaN,NaN,319,243,-0.367614,-0.303614,0.697860,51,trouble (-0.758); comfort (-0.74); acquaint (-...
1,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,NaN,NaN,NaN,421,330,-0.199822,-0.014473,0.818860,59,god's (-1.78); comfort (-0.74); oblig'd (-0.68...
2,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,NaN,NaN,NaN,406,290,-0.456105,-0.281549,0.634597,53,grief (-0.601); pleasure (-0.841); pray (-0.41...
3,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,NaN,NaN,NaN,69,48,-0.689703,-0.655594,0.539041,6,confusion (-0.996); goodness (-1.22); gentleme...
4,"Richardson, Samuel, 1689-1761",Pamela (1741),1741.0,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,NaN,NaN,NaN,373,285,-0.542235,-0.347480,0.810032,52,letter (-0.384); trouble (-0.758); comfort (-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5081,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,2022-12-18 00:00:00,Esq | Soho-Square | London,Esq | Soho-Square | London,232,183,0.044507,0.068118,0.750549,54,compliments (-0.243); gentlemen (-0.375); pres...
5082,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,2022-12-18 00:00:00,Esq | Soho-Square | London,Esq | Soho-Square | London,278,220,0.051787,-0.040459,0.807766,68,inwardly (-0.54); sword (0.528); dropt (1.51);...
5083,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748.0,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,...,2022-12-18 00:00:00,Esq | Soho-Square | London,Esq | Soho-Square | London,273,215,-0.282791,-0.328904,0.711489,57,surgeons (0.163); chevalier (-0.406); colonel ...


In [120]:
richardson_scored.to_pickle('../data/data.absconc_richardson.pkl')